In [2]:
import numpy as np
import steepest_descent_bcktrck as sdb
from steepest_descent_bcktrck import *
import cgm_pol_rib as cgmpb
from cgm_pol_rib import *
import functions as funcs
from sklearn.model_selection import ParameterGrid
from importlib import reload

In [3]:
reload(funcs)
reload(sdb)
reload(cgmpb)

<module 'cgm_pol_rib' from '/home/claudio/Documents/GitHub/Homework-Numerical-Optimisation-/Homework Numerical Optimisation/Unconstrained Optimisation/cgm_pol_rib.py'>

Evaluate rate of convergence

In [4]:
def rate_of_convergence(xseq: np.ndarray) ->np.ndarray:
    N = 100
    n = xseq.shape[1]
    p = np.empty(N)
    for k in range(1, N+1):
        if (xseq[:, k+2]-xseq[:, k+1]).any() != 0 and (xseq[:, k+1]-xseq[:, k]).any() != 0 and (xseq[:, k]-xseq[:, k-1]).any() != 0:
            p[k-1] = log(np.linalg.norm(xseq[:, k+2]-xseq[:, k+1], 2) / (np.linalg.norm(xseq[:, k+1]-xseq[:, k], 2))) / \
                     log(np.linalg.norm(xseq[:, k+1]-xseq[:, k], 2) / (np.linalg.norm(xseq[:, k]-xseq[:, k-1], 2)))
            # print(f"i = {k-1}: p = {p}")
    return p

Initialisation of the data

In [5]:
x0 = np.array([1.2, 1.2]).reshape(-1,1)
x1 = np.array([-1.2, 1]).reshape(-1,1)
alpha0 = 1
tolgrad = 1e-12
rho = 0.5
c = 1e-4
kmax = 50000
btmax = 50
fin_diff = True
fd_type = 'centered'

params = {"c": [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
              "rho": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]}

Let's test the Steepest descent and conjugate gradient method with Backtrack tuning the parameters c and rho using the Armijo condition to see what is the result obtained (POINT X0)

Here our idea is quite simple. In order to make a complete comparison between the two methods the best set of paramenters for both methods is evaluated, once this is done a comparison is made with the best result obtained in one of the two methods with one set and the same set is also used for the other method (we will therefore have in all 2 saved evaluations for steepest descent, one with the best set of parameters, and the other with the best set of parameters of the conjugate gradient method, and 2 saved evaluations for the conjugate gradient method) 


In [11]:
#best result for steepest descent method is saved here
sd_best_fk_x0 = 1
sd_best_param_x0 = 0
sd_best_k_x0 = 0
sd_best_gradfk_norm_x0 = 10
sd_best_xk_x0 = np.empty((2,1))

#best result for conjugate gradient method is saved here
cg_best_fk_x0 = 1
cg_best_param_x0 = 0
cg_best_k_x0 = 0
cg_best_gradfk_norm_x0 = 10
cg_best_xk_x0 = np.empty((2,1))

#Other result of steepest descent method is saved here
sd_fk_x0 = 1
sd_param_x0 = 0
sd_k_x0 = 0
sd_xk_x0 = np.empty((2,1))

#Other result of conjugate gradient method is saved here
cg_fk_x0 = 1
cg_param_x0 = 0
cg_k_x0 = 0
cg__xk_x0 = np.empty((2,1))

for param in ParameterGrid(params): 
    sd_xk, sd_fk, sd_gradfk_norm, sd_k, sd_xseq, sd_btseq = steepest_descent_bcktrck(x0, 'Rosenbrock', alpha0, kmax, tolgrad, param["c"], param["rho"], btmax, fin_diff, fd_type)

    cg_xk, cg_fk, cg_gradfk_norm, cg_k, cg_xseq, cg_btseq = cgm_pol_rib(x0, 'Rosenbrock', alpha0, kmax, tolgrad, param["c"], param["rho"], btmax, fin_diff, fd_type)
    
    if (sd_fk < sd_best_fk_x0): 
        sd_best_param_x0 = param

        sd_best_xk_x0 = sd_xk
        sd_best_k_x0 = sd_k
        sd_best_fk_x0 = sd_fk
        sd_best_gradfk_norm_x0 = sd_gradfk_norm
        sd_best_xseq_x0 = np.copy(sd_xseq)
        
        cg_xk_x0 = cg_xk
        cg_k_x0 = cg_k
        cg_fk_x0 = cg_fk
            
    if (cg_fk < cg_best_fk_x0): 
        cg_best_param_x0 = param

        cg_best_xk_x0 = cg_xk
        cg_best_k_x0 = cg_k
        cg_best_fk_x0 = cg_fk
        cg_best_gradfk_norm_x0 = cg_gradfk_norm
        cg_best_xseq_x0 = np.copy(cg_xseq)

        
        sd_xk_x0 = sd_xk
        sd_k_x0 = sd_k
        sd_fk_x0 = sd_fk

In [12]:
print("Best evaluation with steepest descent method:")
print("x0: ", x0[0], x0[1])
print("best set of parameters: ", sd_best_param_x0)
print("xk: ", sd_best_xk_x0[0], sd_best_xk_x0[1])
print("k: ", sd_best_k_x0)
print("fk: ", sd_best_fk_x0) 

print("\nEvaluation of the conjugate gradient method using the best parameters of steepest descent: ")
print("x0: ", x0[0], x0[1])
print("set of parameters: ", sd_best_param_x0)
print("xk: ", cg_xk_x0[0], cg_xk_x0[1])
print("k: ", cg_k_x0)
print("fk: ", cg_fk_x0) 

Best evaluation with steepest descent method:
x0:  [1.2] [1.2]
best set of parameters:  {'c': 0.01, 'rho': 0.7}
xk:  [1.] [1.]
k:  31839
fk:  [4.17569518e-25]

Evaluation of the conjugate gradient method using the best parameters of steepest descent: 
x0:  [1.2] [1.2]
set of parameters:  0
xk:  [1.] [1.]
k:  8589
fk:  [3.1537594e-25]


Best evaluation with steepest descent method:

x0:  [1.2] [1.2]

best set of parameters:  {'c': 0.01, 'rho': 0.7}

xk:  [1.] [1.]

k:  31839

fk:  [4.17569518e-25]


Evaluation of the conjugate gradient method using the best parameters of steepest descent: 

x0:  [1.2] [1.2]

set of parameters:  {'c': 0.01, 'rho': 0.7}

xk:  [1.] [1.]

k:  8589

fk:  [3.1537594e-25]

In [13]:
print("Best evaluation with conjugate gradient method:")
print("x0: ", x0[0], x0[1])
print("best set of parameters: ", cg_best_param_x0)
print("xk: ", cg_best_xk_x0[0], cg_best_xk_x0[1])
print("k: ", cg_best_k_x0)
print("fk: ", cg_best_fk_x0) 

print("\nEvaluation of the steepest descent method using the best parameters of conjugate gradient: ")
print("x0: ", x0[0], x0[1])
print("set of parameters: ", cg_best_param_x0)
print("xk: ", sd_xk_x0[0], sd_xk_x0[1])
print("k: ", sd_k_x0)
print("fk: ", sd_fk_x0) 


Best evaluation with steepest descent method:
x0:  [1.2] [1.2]
best set of parameters:  {'c': 0.1, 'rho': 0.4}
xk:  [1.] [1.]
k:  330
fk:  [8.72184338e-29]

Evaluation of the conjugate gradient method using the best parameters of steepest descent: 
x0:  [1.2] [1.2]
set of parameters:  0
xk:  [1.] [1.]
k:  27922
fk:  [8.37329555e-25]


Best evaluation with conjugate gradient method:

x0:  [1.2] [1.2]

best set of parameters:  {'c': 0.1, 'rho': 0.4}

xk:  [1.] [1.]

k:  330

fk:  [8.72184338e-29]

Evaluation of the steepest descent method using the best parameters of conjugate gradient: 

x0:  [1.2] [1.2]

set of parameters:  {'c': 0.1, 'rho': 0.4}

xk:  [1.] [1.]

k:  27922

fk:  [8.37329555e-25]

In [14]:
if (sd_best_gradfk_norm_x0 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(sd_best_xseq_x0)
    print("Evaluation of rate of convergence for steepest descent method:")
    print(f"p mean = {np.abs(p).mean():.2f}")

Evaluation of rate of convergence for steepest descent method:
p mean = 1.13


Evaluation of rate of convergence for steepest descent method:

p mean = 1.13

In [15]:
if (cg_best_gradfk_norm_x0 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(cg_best_xseq_x0)
    print("Evaluation of rate of convergence for conjugate gradient method:")
    print(f"p mean = {np.abs(p).mean():.2f}")

Evaluation of rate of convergence for conjugate gradient method:
p mean = 3.78


Evaluation of rate of convergence for conjugate gradient method:

p mean = 3.78

Now let's do the same but using another starting point (x1)

In [16]:
#best result for steepest descent method is saved here
sd_best_fk_x1 = 1
sd_best_param_x1 = 0
sd_best_k_x1 = 0
sd_best_gradfk_norm_x1 = 10
sd_best_xk_x1 = np.empty((2,1))

#best result for conjugate gradient method is saved here
cg_best_fk_x1 = 1
cg_best_param_x1 = 0
cg_best_k_x1 = 0
cg_best_gradfk_norm_x1 = 10
cg_best_xk_x1 = np.empty((2,1)if (sd_best_gradfk_norm_x0 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(sd_best_xseq_x0)
    print("Evaluation of rate of convergence for steepest descent method:")
    print(f"p mean = {np.abs(p).mean():.2f}"))

#Other result of steepest descent method is saved here
sd_fk_x1 = 1
sd_param_x1 = 0
sd_k_x1 = 0
sd_xk_x1 = np.empty((2,1))

#Other result of conjugate gradient method is saved here
cg_fk_x1 = 1
cg_param_x1 = 0
cg_k_x1 = 0
cg__xk_x1 = np.empty((2,1))

for param in ParameterGrid(params): 
    sd_xk, sd_fk, sd_gradfk_norm, sd_k, sd_xseq, sd_btseq = steepest_descent_bcktrck(x1, 'Rosenbrock', alpha0, kmax, tolgrad, param["c"], param["rho"], btmax, fin_diff, fd_type)

    cg_xk, cg_fk, cg_gradfk_norm, cg_k, cg_xseq, cg_btseq = cgm_pol_rib(x1, 'Rosenbrock', alpha0, kmax, tolgrad, param["c"], param["rho"], btmax, fin_diff, fd_type)
    
    if (sd_fk < sd_best_fk_x0): 
        sd_best_param_x1 = param

        sd_best_xk_x1 = sd_xk
        sd_best_k_x1 = sd_k
        sd_best_fk_x1 = sd_fk
        sd_best_gradfk_norm_x1 = sd_gradfk_norm
        sd_best_xseq_x1 = np.copy(sd_xseq)
        
        cg_xk_x1 = cg_xk
        cg_k_x1 = cg_k
        cg_fk_x1 = cg_fk
            
    if (cg_fk < cg_best_fk_x1): 
        cg_best_param_x1 = param

        cg_best_xk_x1 = cg_xk
        cg_best_k_x1 = cg_k
        cg_best_fk_x1 = cg_fk
        cg_best_gradfk_norm_x1 = cg_gradfk_norm
        cg_best_xseq_x1 = np.copy(cg_xseq)

        
        sd_xk_x1 = sd_xk
        sd_k_x1 = sd_k
        sd_fk_x1 = sd_fk

In [18]:
print("Best evaluation with steepest descent method:")
print("x1: ", x1[0], x1[1])
print("best set of parameters: ", sd_best_param_x1)
print("xk: ", sd_best_xk_x1[0], sd_best_xk_x1[1])
print("k: ", sd_best_k_x1)
print("fk: ", sd_best_fk_x1) 

print("\nEvaluation of the conjugate gradient method using the best parameters of steepest descent: ")
print("x0: ", x1[0], x1[1])
print("set of parameters: ", sd_best_param_x1)
print("xk: ", cg_xk_x1[0], cg_xk_x1[1])
print("k: ", cg_k_x1)
print("fk: ", cg_fk_x1) 

Best evaluation with steepest descent method:
x1:  [-1.2] [1.]
best set of parameters:  {'c': 1e-06, 'rho': 0.7}
xk:  [1.] [1.]
k:  35285
fk:  [4.17283211e-25]

Evaluation of the conjugate gradient method using the best parameters of steepest descent: 
x0:  [-1.2] [1.]
set of parameters:  {'c': 1e-06, 'rho': 0.7}
xk:  [1.] [1.]
k:  8191
fk:  [2.04761125e-25]


Best evaluation with steepest descent method:

x1:  [-1.2] [1.]

best set of parameters:  {'c': 1e-06, 'rho': 0.7}

xk:  [1.] [1.]

k:  35285

fk:  [4.17283211e-25]


Evaluation of the conjugate gradient method using the best parameters of steepest descent: 

x0:  [-1.2] [1.]

set of parameters:  {'c': 1e-06, 'rho': 0.7}

xk:  [1.] [1.]

k:  8191

fk:  [2.04761125e-25]

In [19]:
print("Best evaluation with conjugate gradient method:")
print("x0: ", x1[0], x1[1])
print("best set of parameters: ", cg_best_param_x1)
print("xk: ", cg_best_xk_x1[0], cg_best_xk_x1[1])
print("k: ", cg_best_k_x1)
print("fk: ", cg_best_fk_x1) 

print("\nEvaluation of the steepest descent method using the best parameters of conjugate gradient: ")
print("x0: ", x1[0], x1[1])
print("set of parameters: ", cg_best_param_x1)
print("xk: ", sd_xk_x1[0], sd_xk_x1[1])
print("k: ", sd_k_x1)
print("fk: ", sd_fk_x1) 


Best evaluation with conjugate gradient method:
x0:  [-1.2] [1.]
best set of parameters:  {'c': 0.01, 'rho': 0.4}
xk:  [1.] [1.]
k:  318
fk:  [5.77643398e-28]

Evaluation of the steepest descent method using the best parameters of conjugate gradient: 
x0:  [-1.2] [1.]
set of parameters:  {'c': 0.01, 'rho': 0.4}
xk:  [1.] [1.]
k:  28146
fk:  [8.1509053e-25]


Best evaluation with conjugate gradient method:

x0:  [-1.2] [1.]

best set of parameters:  {'c': 0.01, 'rho': 0.4}

xk:  [1.] [1.]

k:  318

fk:  [5.77643398e-28]


Evaluation of the steepest descent method using the best parameters of conjugate gradient: 

x0:  [-1.2] [1.]

set of parameters:  {'c': 0.01, 'rho': 0.4}

xk:  [1.] [1.]

k:  28146

fk:  [8.1509053e-25]

In [20]:
if (sd_best_gradfk_norm_x1 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(sd_best_xseq_x1)
    print("Evaluation of rate of convergence for steepest descent method:")
    print(f"p mean = {np.abs(p).mean():.2f}")

Evaluation of rate of convergence for steepest descent method:
p mean = 1.30


Evaluation of rate of convergence for steepest descent method:

p mean = 1.30

In [21]:
if (cg_best_gradfk_norm_x1 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(cg_best_xseq_x1)
    print("Evaluation of rate of convergence for conjugate gradient method:")
    print(f"p mean = {np.abs(p).mean():.2f}")

Evaluation of rate of convergence for conjugate gradient method:
p mean = 2.18


Evaluation of rate of convergence for conjugate gradient method:

p mean = 2.18